# Assignment 10
A simple single core embedded system executes continuously two tasks: task A and task B.
Both alternates between a running and waiting state, characterized by the following average duration:

| Task | A | B |
| --- | --- | --- |
| Waiting | 10 s. | 5 s. |
| Execution | 8 s. | 12 s. |

Note that, since the system is single core, when both tasks are in execution, they each run at *half of their speed* (the corresponding rate is halved).

## Requests

Considering all waiting and execution times **exponentially distributed** and that the system starts in a state where both tasks are running, compute:

- The utilization and average number of tasks in execution
- The system throughput
- The average number of jobs at t = 10s, t = 20s, t = 50s and t = 100s.

## Drawing the CTMC model and assigning the rewards
The markov chain model is shown in the following picture:



The distributions' rates associated with each edge are:
- $D_{W,A} = \frac{1}{10} s^{-1}$
- $D_{R,A} = \frac{1}{8} s^{-1}$
- $D_{W,B} = \frac{1}{5} s^{-1}$

We can assign some rewards to each state and each edge that will be useful to compute the performance metrics of the system.

For the states, we can consider the rewards *n* and *u* that will be useful, respectively, for the **average number of jobs** in the system and the **utilization** of the system.

| | n | u |
| --- | :-: | :-: |
| $R_{A}, R_{B}$ | 2 | 1 |
| $W_{A},R_{B}$ | 1 | 1 |
| $R_{A},W_{B}$ | 1 | 1 |
| $W_{A},W_{B}$ | 0 | 0 |


In [1]:
n = [2, 1, 1, 0]
u = [1, 1, 1, 0]

The only edge reward we need is *c*, that consists in the number of completions and will be useful for the computation of the **throughput**.

| | $R_{A}, R_{B}$ | $W_{A},R_{B}$ | $R_{A},W_{B}$ | $W_{A},W_{B}$ |
| :--- | :--- | :--- | :--- | :--- |
| $R_{A}, R_{B}$ | 0 | 1 | 1 | 0 |
| $W_{A},R_{B}$ | 0 | 0 | 0 | 1 |
| $R_{A},W_{B}$ | 0 | 0 | 0 | 1 |
| $W_{A},W_{B}$ | 0 | 0 | 0 | 0 |


In [2]:
import numpy as np
c = np.array(
    [[0, 1, 1, 0],
     [0, 0, 0, 1],
     [0, 0, 0, 1],
     [0, 0, 0, 0]])

## Computation of the infinitesimal generator

The infinitesimal generator of the system is:

$$Q = \begin{bmatrix}
(-\frac{1}{8}-\frac{1}{12})*\frac{1}{2} & \frac{1}{8}*\frac{1}{2} & \frac{1}{12}*\frac{1}{2} & 0 \\
\frac{1}{10} & -\frac{1}{10}-\frac{1}{12} & 0 & \frac{1}{12} \\
\frac{1}{5} & 0 & -\frac{1}{5}-\frac{1}{8} & \frac{1}{8} \\
0 & \frac{1}{5} & \frac{1}{10} & -\frac{1}{5}-\frac{1}{10}
\end{bmatrix}$$

---

(the states considered are, in order: $R_{A}, R_{B}$; $W_{A},R_{B}$; $R_{A},W_{B}$ and $W_{A},W_{B}$)

In [3]:
D_wa = 1/10
D_ra = 1/8
D_wb = 1/5
D_rb = 1/12

Q = np.array([
    [ (-D_ra-D_rb)/2,  D_ra/2,      D_rb/2,           0],
    [       D_wa,  -D_wa-D_rb,           0,        D_rb],
    [       D_wb,           0,  -D_wb-D_ra,        D_ra],
    [          0,        D_wb,        D_wa,  -D_wb-D_wa]
])

## Steady state and transient probabilities

Here we have now to compute the probabilities of the system being in each state, both at steady state and at the time requested by the exercise.

### Steady state

In [4]:
u_init = [1, 0, 0, 0]

Q_steady = Q.copy()
Q_steady[:,0] = [1, 1, 1, 1]

ss_probabilities = np.matmul(u_init, np.linalg.inv(Q_steady))

In [5]:
print("The steady state probabilities found are:\n")
print("Ra,Rb\t{:%}".format(ss_probabilities[0]))
print("Wa,Rb\t{:%}".format(ss_probabilities[1]))
print("Ra,Wb\t{:%}".format(ss_probabilities[2]))
print("Wa,Wb\t{:%}".format(ss_probabilities[3]))

The steady state probabilities found are:

Ra,Rb	47.761194%
Wa,Rb	29.850746%
Ra,Wb	9.950249%
Wa,Wb	12.437811%


### Transient
The transient probabilities must be calculated at times t = 10s, t = 20s, t = 50s and t = 100s.

In [6]:
from scipy import linalg as slin

times = [10, 20, 50, 100]

p0=[1, 0, 0, 0]
tr_probabilities = []
for t in times:
    tr_probabilities.append(np.matmul(p0, slin.expm(Q*t)))
    
print("The transient probabilities found are:\n")
for t,p in zip(times, tr_probabilities):
    print("time: {}s".format(t))
    print("\tRa,Rb\t{:%}".format(p[0]))
    print("\tWa,Rb\t{:%}".format(p[1]))
    print("\tRa,Wb\t{:%}".format(p[2]))
    print("\tWa,Wb\t{:%}".format(p[3]))

The transient probabilities found are:

time: 10s
	Ra,Rb	56.663490%
	Wa,Rb	24.497398%
	Ra,Wb	9.820043%
	Wa,Wb	9.019069%
time: 20s
	Ra,Rb	49.461861%
	Wa,Rb	28.774378%
	Ra,Wb	9.973839%
	Wa,Wb	11.789922%
time: 50s
	Ra,Rb	47.773519%
	Wa,Rb	29.842736%
	Ra,Wb	9.950561%
	Wa,Wb	12.433184%
time: 100s
	Ra,Rb	47.761197%
	Wa,Rb	29.850744%
	Ra,Wb	9.950249%
	Wa,Wb	12.437810%


## Utilization (at steady state)

The steady state utilization can be computed starting from the state reward **u**.

In [7]:
U = np.sum(ss_probabilities * u)

print("The utilization of the system at the steady state is {:.4f}".format(U))

The utilization of the system at the steady state is 0.8756


## Number of jobs (at steady state)

The the number of jobs at steady state can be computed starting from the state reward **n**.

In [8]:
N = np.sum(ss_probabilities * n)

print("The number of jobs in the system at the steady state is {:.4f}".format(N))

The number of jobs in the system at the steady state is 1.3532


## System throughput (at steady state)
The system throughput at steady state can be computed starting from the edge reward **c**.

In [9]:
C = c*Q
x = np.sum(c*Q,axis=1)
X = np.sum(ss_probabilities * x)
print("The throughput of the system at the steady state is {:.4f}".format(X))

The throughput of the system at the steady state is 0.0871


## The average number of jobs (at transient)
The average number of jobs at transient time can be computed from the state reward **n** and the transient probabilities.

In [10]:
print("The number of jobs in the system for each transient state is:\n")
for t,p in zip(times, tr_probabilities):
    nt = np.sum(p * n)
    print("t = {}s \t N = {:.4f}".format(t,nt))

The number of jobs in the system for each transient state is:

t = 10s 	 N = 1.4764
t = 20s 	 N = 1.3767
t = 50s 	 N = 1.3534
t = 100s 	 N = 1.3532
